Hello and Welcome to Data_Merge the file where I take some csv's and merge them into a large csv that suits our needs. 
## Step 1: Importing character Metadata
In the previous file we created and edited a file of character metadata, import that here. 

In [1]:
import pandas as pd
character_data = pd.read_csv("character_metadata.csv", index_col=None, header=0)
character_data.head()

,character id,name,id number,title,gender,billing order
0,u0,BIANCA,m0,10 things i hate about you,f,4
1,u1,BRUCE,m0,10 things i hate about you,?,?
2,u2,CAMERON,m0,10 things i hate about you,m,3
3,u3,CHASTITY,m0,10 things i hate about you,?,?
4,u4,JOEY,m0,10 things i hate about you,m,6


## Step 2: Importing the conversations file
This txt file is very large so you need to change your jupyter notebook settings if you get an error running this code. 

Here's a link to a tutorial on how to fix the IOPub error. 
https://www.drjamesfroggatt.com/python-and-neural-networks/iopub-data-rate-exceeded-the-notebook-server-will-temporarily-stop-sending-output-to-the-client-in-order-to-avoid-crashing-it/

In [2]:
list_ = open("movie_conversations.txt").read().split(" +++$+++ ")
# print(list_)
new_list = []
for i in range(len(list_)):
    new_list.extend(list_[i].split("\n"))

Then we reformat the list of values as a dataframe

In [3]:
id1 = new_list[0:-1:4]
id2 = new_list[1: :4]
movie = new_list[2: :4]
movie_lines = new_list[3: :4]
d = {'id1': id1, 'id2': id2, 'movie_id': movie, 'lines': movie_lines}
convo_metadata = pd.DataFrame(data=d)

## Step 3: Adding Character Gender to the dataset
So far we have a dataframe of conversations but we want to be able to filter it so it's only F/F conversations. To do so we have to add the gender of the characters speaking. Which we do by merging the character metadata and conversation dataset.

In [4]:
id1_set = convo_metadata.merge(character_data, left_on='id1', right_on='character id')
full_set = id1_set.merge(character_data, left_on='id2', right_on='character id')
print(full_set)

         id1    id2 movie_id                                         lines  \
0         u0     u2       m0              ['L194', 'L195', 'L196', 'L197']   
1         u0     u2       m0                              ['L198', 'L199']   
2         u0     u2       m0              ['L200', 'L201', 'L202', 'L203']   
3         u0     u2       m0                      ['L204', 'L205', 'L206']   
4         u0     u2       m0                              ['L207', 'L208']   
...      ...    ...      ...                                           ...   
83092  u9028  u9031     m616  ['L666324', 'L666325', 'L666326', 'L666327']   
83093  u9028  u9031     m616                        ['L666575', 'L666576']   
83094  u9030  u9034     m616                        ['L666256', 'L666257']   
83095  u9030  u9034     m616  ['L666369', 'L666370', 'L666371', 'L666372']   
83096  u9030  u9034     m616             ['L666520', 'L666521', 'L666522']   

      character id_x    name_x id number_x                     

## Step 4: Filtering on F/F
Here I clean up the dataset a little and then filter based on gender of id1 and id2. 

In [124]:
d = {'id1': full_set['id1'], 'id2': full_set['id2'], 'id1_gender': full_set['gender_x'], 'id2_gender': full_set['gender_y'], 'movie_id': full_set['movie_id'], 'lines': full_set['lines']}
clean_set = pd.DataFrame(data=d)
#print(clean_set)
id1f = clean_set[clean_set['id1_gender']=='f']
bechdel = id1f[id1f['id2_gender']=='f'].reset_index()
bechdel.head()

,index,id1,id2,id1_gender,id2_gender,movie_id,lines
0,38,u0,u5,f,f,m0,"['L164', 'L165']"
1,39,u0,u5,f,f,m0,"['L319', 'L320']"
2,40,u0,u5,f,f,m0,"['L441', 'L442', 'L443', 'L444', 'L445']"
3,41,u0,u5,f,f,m0,"['L525', 'L526', 'L527']"
4,42,u0,u5,f,f,m0,"['L529', 'L530']"


In [66]:
list_ = open("movie_lines.txt").read().split(" +++$+++ ")

new_list = []
for i in range(len(list_)):
    new_list.extend(list_[i].split("\n"))

In [120]:
line_id = new_list[0:-1:5]
movie_line = new_list[4: :5]
line_data = dict(zip(line_id, movie_line))

In [107]:

convo = bechdel['lines']
print(type(convo[1]))

every_tag = []
for z in range(len(convo)):
    convo[z] = convo[z].replace("'", "")
    convo[z] = convo[z].replace("[", "")
    convo[z] = convo[z].replace("]", "")
    convo[z] = convo[z].split(", ")
    every_tag.append(convo[z])


<class 'list'>


AttributeError: 'list' object has no attribute 'replace'

In [121]:
text_line = []

for j in range(len(convo)):
    ce = convo[j]
    for k in range(len(ce)):
            linetext = line_data.get(ce[k])
            if k == 0:
                linex = [linetext]
            else: 
                linex.append(linetext)
    text_line.append(linex)

print(text_line)

[["Where've you been?", 'Nowhere... Hi, Daddy.'], ["I have the potential to smack the crap out of you if you don't get out of my way.", 'Can you at least start wearing a bra?'], ["Oh my God, does this mean you're becoming normal?", "It means that Gigglepuss is playing at Club Skunk and we're going.", "Oh, I thought you might have a date  I don't know why I'm bothering to ask, but are you going to Bogey Lowenstein's party Saturday night?", 'What do you think?', "I think you're a freak.  I think you do this to torture me.  And I think you suck."], ["You're ruining my life'  Because you won't be normal, I can't be normal.", "What's normal?", "Bogey Lowenstein's party is normal, but you're too busy listening to Bitches Who Need Prozac to know that."], ["Can't you forget for just one night that you're completely wretched?", "At least I'm not a clouted fen- sucked hedge-pig."], ["Like I'm supposed to know what that even means.", "It's Shakespeare.  Maybe you've heard of him?", "Yeah, he's yo

In [125]:
bechdel['lines_text'] = text_line
bechdel.head()

,index,id1,id2,id1_gender,id2_gender,movie_id,lines,lines_text
0,38,u0,u5,f,f,m0,"['L164', 'L165']","[Where've you been?, Nowhere... Hi, Daddy.]"
1,39,u0,u5,f,f,m0,"['L319', 'L320']",[I have the potential to smack the crap out of...
2,40,u0,u5,f,f,m0,"['L441', 'L442', 'L443', 'L444', 'L445']","[Oh my God, does this mean you're becoming nor..."
3,41,u0,u5,f,f,m0,"['L525', 'L526', 'L527']",[You're ruining my life' Because you won't be...
4,42,u0,u5,f,f,m0,"['L529', 'L530']",[Can't you forget for just one night that you'...


In [126]:
movie_data = pd.read_csv("movie_metadata.csv", index_col=None, header=0)
movie_data.head()

,id number,title,year of release,imdb rating,number of reviewers,genre(s)
0,m0,10 things i hate about you,1999,6.9,62847,"['comedy', 'romance']"
1,m1,1492: conquest of paradise,1992,6.2,10421,"['adventure', 'biography', 'drama', 'history']"
2,m2,15 minutes,2001,6.1,25854,"['action', 'crime', 'drama', 'thriller']"
3,m3,2001: a space odyssey,1968,8.4,163227,"['adventure', 'mystery', 'sci-fi']"
4,m4,48 hrs.,1982,6.9,22289,"['action', 'comedy', 'crime', 'drama', 'thrill..."


In [130]:
bechdel.merge(movie_data, left_on = "movie_id", right_on = "id number", copy = False)

,index,id1,id2,id1_gender,id2_gender,movie_id,lines,lines_text,id number,title,year of release,imdb rating,number of reviewers,genre(s)
0,38,u0,u5,f,f,m0,"['L164', 'L165']","[Where've you been?, Nowhere... Hi, Daddy.]",m0,10 things i hate about you,1999,6.9,62847,"['comedy', 'romance']"
1,39,u0,u5,f,f,m0,"['L319', 'L320']",[I have the potential to smack the crap out of...,m0,10 things i hate about you,1999,6.9,62847,"['comedy', 'romance']"
2,40,u0,u5,f,f,m0,"['L441', 'L442', 'L443', 'L444', 'L445']","[Oh my God, does this mean you're becoming nor...",m0,10 things i hate about you,1999,6.9,62847,"['comedy', 'romance']"
3,41,u0,u5,f,f,m0,"['L525', 'L526', 'L527']",[You're ruining my life' Because you won't be...,m0,10 things i hate about you,1999,6.9,62847,"['comedy', 'romance']"
4,42,u0,u5,f,f,m0,"['L529', 'L530']",[Can't you forget for just one night that you'...,m0,10 things i hate about you,1999,6.9,62847,"['comedy', 'romance']"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2815,82375,u8950,u8951,f,f,m610,"['L660446', 'L660447', 'L660448']","[Oh, will you help me? Can you help me?, You ...",m610,the wizard of oz,1939,8.3,104873,"['adventure', 'family', 'fantasy', 'musical']"
2816,82376,u8950,u8951,f,f,m610,"['L660458', 'L660459', 'L660460', 'L660461', '...","[...Toto, too?, Toto, too., Oh, now?, Whenever...",m610,the wizard of oz,1939,8.3,104873,"['adventure', 'family', 'fantasy', 'musical']"
2817,82377,u8950,u8951,f,f,m610,"['L660467', 'L660468']","[Are you ready now?, Yes. Say goodbye, Toto.]",m610,the wizard of oz,1939,8.3,104873,"['adventure', 'family', 'fantasy', 'musical']"
2818,82378,u8950,u8951,f,f,m610,"['L660469', 'L660470']","[Yes, I'm ready now., Then close your eyes, an...",m610,the wizard of oz,1939,8.3,104873,"['adventure', 'family', 'fantasy', 'musical']"
